<a href="https://colab.research.google.com/github/donghaim12/REARGUARD/blob/master/retrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
from google.colab import drive 

drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
tf.test.gpu_device_name()
!nvidia-smi

Thu Aug 13 00:23:33 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    27W /  70W |    227MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.callbacks import *
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten
from tensorflow.keras.applications.mobilenet import MobileNet
from sklearn.model_selection import StratifiedKFold
import keras

kfold = StratifiedKFold(n_splits=5, shuffle=True)

train_dir = '/content/gdrive/My Drive/binary_training'
validation_dir = '/content/gdrive/My Drive/binary_test'

pre_trained_model = InceptionV3(input_shape=(224,224,3),
                                include_top=False,
                                weights='imagenet')

# pre_trained_model2 = MobileNet(input_shape=())

for layer in pre_trained_model.layers:
    layer.trainable = False

add_model = Sequential()
add_model.add(pre_trained_model)
add_model.add(GlobalAveragePooling2D())
add_model.add(Dense(2, activation = 'softmax'))


model = add_model


model.compile(optimizer= Adam(lr=0.001),
              loss = 'binary_crossentropy',
              metrics= ['acc'])

model.summary()

train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   horizontal_flip = True)


test_datagen = ImageDataGenerator(rescale = 1.0/255.)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size = (224,224),
                                                    batch_size = 32,
                                                    shuffle = True)

validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                        target_size = (224,224),
                                                        batch_size = 32,
                                                        shuffle = False)


earlystop = EarlyStopping(monitor='val_loss',
                          mode = 'min',
                          patience=10,
                          verbose=1
                          )


history = model.fit_generator(
    generator=train_generator,
    steps_per_epoch = len(train_generator)//32,
    validation_data = validation_generator,
    validation_steps = len(validation_generator),
    epochs = 300,
    callbacks = [earlystop]
)


acc = history.history['acc']
val_acc = history.history.get('val_acc')
loss = history.history['loss']
val_loss = history.history.get('val_loss')

epochs = range(len(acc))


plt.subplot(1, 2, 1)
plt.title('Training and validation accuracy')
plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.legend()

#plt.figure()
plt.subplot(1, 2, 2)
plt.title('Training and validation loss')
plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.legend()

plt.show()

from keras.models import load_model
model.save('/content/gdrive/My Drive/BCmodel.h5')

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 5, 5, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 4098      
Total params: 21,806,882
Trainable params: 4,098
Non-trainable params: 21,802,784
_________________________________________________________________


FileNotFoundError: ignored

In [ ]:
import shutil

Found 5195 images belonging to 2 classes.
Found 1309 images belonging to 2 classes.


ValueError: ignored

ValueError: ignored